In [1]:
#Importando Bibliotecas
import pandas as pd
from datetime import timedelta
import scipy.stats as stats

In [2]:
#Importando as Bases
df_full = pd.read_csv('5.SERV_FALEC_EXON.csv', sep=';', encoding='UTF-8').assign(IDADE = '')

df_full['DT_NASC_SERVIDOR'] = pd.to_datetime(df_full['DT_NASC_SERVIDOR'], yearfirst=True, format='%d/%m/%Y')
df_full['DT_BASE'] = '31/12/2020'
df_full['DT_BASE'] = pd.to_datetime(df_full.DT_BASE, yearfirst=True, format='%d/%m/%Y')
df_full['IDADE'] = round((df_full.DT_BASE - df_full.DT_NASC_SERVIDOR) / timedelta(days=365.25) - 0.5).astype(int)

In [3]:
#Importando as Tábuas
df_tabuas = pd.read_excel('Funções Biométricas (Com Fórmulas).xlsx', sheet_name='qx')

In [6]:
ary_expostos = []
ary_esperados = []
for l in range(126+1):
    linha = []
    linha2 = []
    #Criando uma matriz com a quantidade de registros de cada IDADE
    for c in range(1):
        qtda = len(df_full[df_full['IDADE'] == l]) #Quantidade de eventos ocorridos em cada idade
        linha.append(qtda)
    ary_expostos.append(linha)

    #Criando uma matriz com o número de MORTES esperadas por idade e o X² calculado
    for c2 in range(len(df_tabuas.columns)):
        esperado = ary_expostos[l][0] * df_tabuas.iloc[l, c2] #Expostos * qx da tábua
        linha2.append(esperado)
    ary_esperados.append(linha2)

df_expostos = pd.DataFrame(data=ary_expostos)
df_esperados = pd.DataFrame(data=ary_esperados, columns=df_tabuas.columns.values)

In [5]:
#Cálculo do X²
ary_qui_quadrado = []
for l3 in range(127):
    linha3 = []
    for c3 in range(len(df_tabuas.columns)):
        if df_esperados.iloc[l3, c3] == 0.0:
            qui_quadrado = 0
            linha3.append(qui_quadrado)
        elif df_esperados.iloc[l3, c3] != 0.0:
            qui_quadrado = (ary_expostos[l3][0] - df_esperados.iloc[l3, c3])**2/df_esperados.iloc[l3, c3]
            linha3.append(qui_quadrado)
    ary_qui_quadrado.append(linha3)

df_qui_quadrado = pd.DataFrame(data=ary_qui_quadrado, columns=df_tabuas.columns.values)
df_qui_quadrado.loc['qui_quadrado'] = round(df_qui_quadrado.sum(axis=0), 2)
df_qui_quadrado.loc['p_value'] = 1-stats.chi2.cdf(x=df_qui_quadrado.loc['qui_quadrado'], df = int(df_expostos[df_expostos != 0].count() - 1))
qui_quadrado_tabu = stats.chi2.ppf(q = 0.95, df = int(df_expostos[df_expostos != 0].count() - 1))

In [ ]:
#Cálculo do Kolmogorov-Smirnov K-S
ary_k_s = []
for l4 in range(127):
    linha4 = []
    for c4 in range(len(df_tabuas.columns)):
        #try:
       ary_k_s = df_expostos.loc[:l4].sum() / df_expostos.sum()
        #except ZeroDivisionError:      
         
ary_k_s
#df_k_s = pd.DataFrame(data=ary_k_s, columns=df_tabuas.columns.values)
#df_k_s.loc['k_s'] = round(df_k_s.sum(axis=0), 2)
#df_k_s.loc['p_value'] = 1-stats.chi2.cdf(x=df_k_s.loc['k_s'], df = int(df_expostos[df_expostos != 0].count() - 1))

#qui_quadrado_tabu = stats.chi2.ppf(q = 0.95, df = int(df_expostos[df_expostos != 0].count() - 1))
#df_qui_quadrado

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats

'''ary = np.array([[6,1], [1.409100,1.523520]])
chi2_contingency(ary)

print(chi2_contingency(ary))

#15,137217'''

qui_quadrado_tabu = stats.chi2.ppf(q = 0.95, df = int(df_expostos[df_expostos != 0].count() - 1))
#p_valor = 1-stats.chi2.cdf(x=18.194805, df=4)
p_valor = round(1-stats.chi2.cdf(x=39.29, df= 19), 6)
p_balor_k_S = stats.k

print('Critical Value')
print(qui_quadrado_tabu)
print(p_valor)

In [109]:
#Cálculo do Kolmogorov-Smirnov K-S

df_esperados_acum = df_esperados.cumsum()
df_expostos_acum = df_expostos.cumsum()
ary_k_s = []
for l in range(126+1):
    linha2 = []
    for c2 in range(len(df_tabuas.columns)):
        k_s_calculado =  round(abs(((float(df_expostos_acum.loc[l]) / int(df_expostos.sum()))) - (float(df_esperados_acum.iloc[l, c2]) / df_esperados.iloc[:,c2].sum())), 6)
        linha2.append(k_s_calculado)
    ary_k_s.append(linha2)

df_k_s = pd.DataFrame(data=ary_k_s, columns=df_tabuas.columns.values)

In [100]:
df_k_s['AMERICAN EXPERIENCE'].sum()

131336.0

In [76]:
df_esperados_acum.iloc[22:27,0]

22     1.3
23     5.2
24    11.7
25    22.1
26    27.3
Name: ALLG-72, dtype: float64

In [37]:
df_expostos_acum.iloc[2, 2]
#df_esperados['AMERICAN EXPERIENCE'].sum()

IndexError: single positional indexer is out-of-bounds

In [ ]:
#Cálculo do Kolmogorov-Smirnov K-S
ary_k_s = []
for l4 in range(127):
    linha4 = []
    for c4 in range(len(df_tabuas.columns)):
        try:
            ary_k_s = ary_expostos.loc[:l4].sum() / df_expostos.sum()
        except ZeroDivisionError:      
         
        
        if df_esperados.iloc[l4, c4] != 0.0:
            qui_k_s = (ary_expostos[l4][0] - df_esperados.iloc[l4, c4])**2/df_esperados.iloc[l4, c4]
            linha4.append(qui_k_s)
    ary_k_s.append(linha4)

for c in range(127):
    Nx_morta.loc[c] = Dx_morta.loc[c:].sum()#.map('{:2f}'.format)

df_k_s = pd.DataFrame(data=ary_k_s, columns=df_tabuas.columns.values)
df_k_s.loc['k_s'] = round(df_k_s.sum(axis=0), 2)
df_k_s.loc['p_value'] = 1-stats.chi2.cdf(x=df_k_s.loc['k_s'], df = int(df_expostos[df_expostos != 0].count() - 1))

qui_quadrado_tabu = stats.chi2.ppf(q = 0.95, df = int(df_expostos[df_expostos != 0].count() - 1))
df_qui_quadrado